In [769]:
import numpy as np
import pandas as pd

DIR = "/Users/rithikpothuganti/quantsc/Sports-Betting"
START_YEAR = 2002
END_YEAR = 2023


In [770]:
team_stats = pd.read_csv(f"{DIR}/data/NBA_team_stats/{START_YEAR}/team_stats.csv", usecols=lambda column: column.startswith('Unnamed') == False)
playoff_wins = pd.read_csv(f"{DIR}/data/NBA_playoff_wins/{START_YEAR}/playoff_wins.csv", usecols=lambda column: column.startswith('Unnamed') == False)
for year in range(START_YEAR+1, END_YEAR):
    playoff_wins_curr = pd.read_csv(f"{DIR}/data/NBA_playoff_wins/{year}/playoff_wins.csv", usecols=lambda column: column.startswith('Unnamed') == False)
    playoff_wins = pd.concat([playoff_wins, playoff_wins_curr])
    team_stats_curr = pd.read_csv(f"{DIR}/data/NBA_team_stats/{year}/team_stats.csv", usecols=lambda column: column.startswith('Unnamed') == False)
    team_stats = pd.concat([team_stats, team_stats_curr])

data = pd.merge(team_stats, playoff_wins, how='left').fillna(-1)

data


,TEAM,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,opp_ORB,opp_DRB,opp_TRB,opp_AST,opp_STL,opp_BLK,opp_TOV,opp_PF,opp_PTS,PLAYOFF_WINS
0,SAC_2002,27.0,61.0,21.0,61,21,7.61,0.00,7.61,109.0,...,13.2,33.2,46.4,20.6,8.8,5.8,17.4,22.0,99.5,10.0
1,LAL_2002,27.9,58.0,24.0,60,22,7.12,0.03,7.15,109.4,...,13.5,32.6,46.1,20.7,8.5,5.7,16.0,25.5,99.7,15.0
2,SAS_2002,28.5,58.0,24.0,59,23,6.21,0.07,6.28,106.5,...,12.3,34.2,46.5,20.1,8.5,4.9,15.6,23.5,100.2,4.0
3,NJN_2002,26.1,52.0,30.0,53,29,4.16,-0.49,3.67,104.0,...,13.4,32.1,45.6,24.2,9.0,6.3,16.5,25.0,100.3,11.0
4,DAL_2002,27.6,57.0,25.0,53,29,4.26,0.16,4.41,112.2,...,12.5,36.7,49.2,23.6,8.2,6.2,17.5,22.7,101.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,DET_2022,23.6,23.0,59.0,22,60,-7.72,0.37,-7.36,106.0,...,10.4,34.4,44.8,26.0,7.2,4.6,13.0,20.8,114.9,-1.0
623,ORL_2022,23.3,22.0,60.0,21,61,-8.00,0.33,-7.67,104.5,...,10.5,35.1,45.6,25.2,8.1,4.8,13.4,20.2,115.3,-1.0
624,OKC_2022,22.4,24.0,58.0,21,61,-8.10,0.20,-7.90,104.6,...,10.3,33.3,43.5,25.4,8.2,4.9,13.4,19.4,116.1,-1.0
625,HOU_2022,24.1,20.0,62.0,21,61,-8.48,0.22,-8.26,108.4,...,11.0,34.1,45.1,25.3,9.3,5.7,13.9,21.8,116.7,-1.0


In [771]:
# Linear Regression

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [772]:
# Define the features and the target
X = data.drop(columns=['TEAM', 'PLAYOFF_WINS'])
# for col in X.columns:
#     col_mean = X[col].mean()
#     col_min = X[col].min()
#     col_max = X[col].max()
#     X[col] = (X[col] - col_mean) / (col_max - col_min)
#     X[col] = X[col] * 2 - 1
y = data['PLAYOFF_WINS']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X = X.fillna(0)


In [773]:
# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [774]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model using mean squared error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error: ", mse)
print("R-squared: ", r2)

Mean Squared Error:  10.70177374124233
R-squared:  0.40525097257328124


In [775]:
# Neural Network
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F


In [776]:
class NBAPlayoffPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NBAPlayoffPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x


In [777]:
# Define the features and the target
X = data.drop(columns=['TEAM', 'PLAYOFF_WINS']).values
y = data['PLAYOFF_WINS'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader objects
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_data = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [778]:
# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
model = NBAPlayoffPredictor(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model
patience = 100
early_stop_counter = 0
best_val_loss = float('inf')
epochs = 1000

for epoch in range(epochs):
    # Training loop
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to('cpu'), targets.to('cpu')
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print(f"Early stopping triggered. Stopping training at epoch {epoch+1}.")
        break

Epoch 1, Validation Loss: 18.5157
Epoch 2, Validation Loss: 18.4376
Epoch 3, Validation Loss: 18.2599
Epoch 4, Validation Loss: 18.1609
Epoch 5, Validation Loss: 18.0439
Epoch 6, Validation Loss: 17.8800
Epoch 7, Validation Loss: 17.7033
Epoch 8, Validation Loss: 17.5925
Epoch 9, Validation Loss: 17.3540
Epoch 10, Validation Loss: 17.1122
Epoch 11, Validation Loss: 16.9598
Epoch 12, Validation Loss: 16.8295
Epoch 13, Validation Loss: 16.6692
Epoch 14, Validation Loss: 16.5090
Epoch 15, Validation Loss: 16.2739
Epoch 16, Validation Loss: 15.9759
Epoch 17, Validation Loss: 15.8163
Epoch 18, Validation Loss: 15.6180
Epoch 19, Validation Loss: 15.3297
Epoch 20, Validation Loss: 15.1786
Epoch 21, Validation Loss: 14.9781
Epoch 22, Validation Loss: 14.7431
Epoch 23, Validation Loss: 14.5175
Epoch 24, Validation Loss: 14.3797
Epoch 25, Validation Loss: 14.2453
Epoch 26, Validation Loss: 14.0036
Epoch 27, Validation Loss: 13.5773
Epoch 28, Validation Loss: 13.3549
Epoch 29, Validation Loss: 13

In [779]:
model.eval()
with torch.no_grad():
    test_loss = 0


In [780]:

model.eval()

# Initialize a variable to store the test loss
test_loss = 0

# No need to compute gradients during evaluation, so use torch.no_grad()
with torch.no_grad():
    for inputs, targets in test_loader:
        # Move inputs and targets to the appropriate device (CPU or GPU)
        inputs, targets = inputs.to('cpu'), targets.to('cpu')

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        # Accumulate the test loss
        test_loss += loss.item()

# Compute the average test loss
average_test_loss = test_loss / len(test_loader)

# Print the average test loss
print("Test Loss: {:.4f}".format(average_test_loss))


Test Loss: 6.1958


In [781]:
def r_squared(y_true, y_pred):
    ssr = torch.sum((y_true - y_pred)**2)
    sst = torch.sum((y_true - torch.mean(y_true))**2)
    r2 = 1 - (ssr / sst)
    return r2.item()

# Assuming the following variables are defined:
# val_loader: DataLoader for your validation or test dataset
# model: your trained neural network model
# device: the device to use (e.g., 'cpu' or 'cuda')

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to('cpu'), targets.to('cpu')
        outputs = model(inputs)
        y_true.append(targets)
        y_pred.append(outputs)

y_true = torch.cat(y_true).detach()
y_pred = torch.cat(y_pred).detach()

r2 = r_squared(y_true, y_pred)
print(f"R-squared value: {r2:.4f}")


R-squared value: 0.6044


In [782]:
# K Nearest Neighbors

from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=35)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

mse_knn = mean_squared_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)

print("KNN Mean Squared Error: ", mse_knn)
print("KNN R-squared: ", r2_knn)

KNN Mean Squared Error:  7.059954648526077
KNN R-squared:  0.5454094743672939


In [783]:
# SVM 

from sklearn.svm import SVR

svm = SVR(kernel='rbf', C=1)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

mse_svm = mean_squared_error(y_test, y_pred_svm)
r2_svm = r2_score(y_test, y_pred_svm)

print("SVM Mean Squared Error: ", mse_svm)
print("SVM R-squared: ", r2_svm)

SVM Mean Squared Error:  6.6071854697970425
SVM R-squared:  0.5745632847238082


In [811]:
# Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Mean Squared Error: ", mse_rf)
print("Random Forest R-squared: ", r2_rf)

Random Forest Mean Squared Error:  6.149430952380952
Random Forest R-squared:  0.6040381010707334


array([-1.  ,  1.74, -0.51, -1.  ,  1.16,  2.4 , -0.98,  1.11, -0.57,
       -0.81,  9.38, -1.  , -0.55,  2.52, -0.96,  9.5 ,  2.93, -0.94,
        4.06, -1.  ,  4.51,  1.9 , -1.  , -0.83,  8.21, -0.96,  5.03,
        4.19,  1.83, -0.03, -1.  , -1.  ,  8.55, -1.  , -0.38,  2.52,
       -1.  , -1.  ,  7.84,  2.9 ,  6.91, -0.78,  4.61, -1.  ,  6.23,
       -0.24,  3.11,  4.76,  9.59,  3.13, -0.86, -0.92,  1.43, -0.9 ,
        7.32, -0.8 ,  1.44, -0.95, -1.  ,  1.17, -1.  , -1.  ,  2.05,
       -0.95, -0.09,  7.73, -1.  , -0.17, -0.97, -0.98, -0.95,  8.51,
       -0.99, -0.5 ,  8.43,  3.7 , -1.  , -1.  , -0.21, -0.78, -1.  ,
        2.08, -0.57,  9.19,  6.46, -0.8 ,  0.83,  6.59, -1.  , -1.  ,
       -1.  ,  7.98,  7.13,  8.87,  1.91, -0.35,  6.57,  2.16,  3.07,
        2.47,  1.6 ,  9.14,  2.17, -0.77, -1.  ,  2.87,  1.32,  6.81,
       10.43,  0.93, -0.97,  2.39, -0.55,  2.96,  1.51, -1.  , -0.65,
        1.16,  2.22,  9.37, -0.58, -1.  , -0.86,  4.03,  4.32,  2.21])